<a href="https://colab.research.google.com/github/choons1k2/2022-Fall-Opensource-AI/blob/main/gender_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
f_path = '/content/heart.csv'

import pandas as pd
heart = pd.read_csv(f_path)


#OneHotEncoding
heart = pd.get_dummies(heart)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


for i in range(20):

  Male = heart[ heart['Sex_M'] == 1 ]
  Female = heart[ heart['Sex_M'] == 0 ]
  Male = Male.sample(frac=0.266)
  


  heartTest = pd.concat([Male, Female])
  print(f"{i+1}. 남성 표본: {len(Male)}명, 여성 표본: {len(Female)}명")
  Positive = heartTest[ heartTest['HeartDisease'] == 1 ]
  Negative = heartTest[ heartTest['HeartDisease'] == 0 ]

  mPositive = len(Positive[Positive['Sex_M'] == 1])
  fPositive = len(Positive[Positive['Sex_M'] == 0])

  mNegative = len(Negative[Negative['Sex_M'] == 1])
  fNegative = len(Negative[Negative['Sex_M'] == 0])

  mRate = mPositive / (mPositive + mNegative)
  fRate = fPositive / (fPositive + fNegative)
  
  print(f"남성 양성률: {mRate}")
  print(f"여성 양성률: {fRate}\n")

1. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6010362694300518
여성 양성률: 0.25906735751295334

2. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6321243523316062
여성 양성률: 0.25906735751295334

3. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6683937823834197
여성 양성률: 0.25906735751295334

4. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6321243523316062
여성 양성률: 0.25906735751295334

5. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6113989637305699
여성 양성률: 0.25906735751295334

6. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6217616580310881
여성 양성률: 0.25906735751295334

7. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.5906735751295337
여성 양성률: 0.25906735751295334

8. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.616580310880829
여성 양성률: 0.25906735751295334

9. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6580310880829016
여성 양성률: 0.25906735751295334

10. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.5906735751295337
여성 양성률: 0.25906735751295334

11. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.5906735751295337
여성 양성률: 0.25906735751295334

12. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6113989637305699
여성 양성률: 0.259067357

In [ ]:
Male = heart[ heart['Sex_M'] == 1 ]
Female = heart[ heart['Sex_M'] == 0 ]
Male = Male.sample(frac=0.266, random_state=123)

MaleX = Male.drop('HeartDisease', axis=1)
Maley = Male['HeartDisease']


FemaleX = Female.drop('HeartDisease', axis=1)
Femaley = Female['HeartDisease']

from sklearn.model_selection import train_test_split
Male_X_train, Male_X_test, Male_y_train, Male_y_test = train_test_split(MaleX, Maley, 
                                                    test_size=0.2, 
                                                    random_state=123)
Female_X_train, Female_X_test, Female_y_train, Female_y_test = train_test_split(FemaleX, Femaley, 
                                                    test_size=0.2, 
                                                    random_state=123)
Total_X_train = pd.concat([Male_X_train, Female_X_train])
Total_X_test = pd.concat([Male_X_test, Female_X_test])
Total_y_train = pd.concat([Male_y_train, Female_y_train]) 
Total_y_test = pd.concat([Male_y_test, Female_y_test])

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import BinaryAccuracy
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [ ]:
model = Sequential()


model.add(Dense(units=20,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=60,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=30,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=1,activation='sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Adam(learning_rate=0.001),
              metrics=[BinaryAccuracy()])

In [ ]:
model.fit(Total_X_train, Total_y_train, epochs = 250, 
          batch_size = 30, 
          validation_data = (Total_X_test, Total_y_test))

Epoch 1/250
11/11 [==============================] - 0s 13ms/step - loss: 0.4315 - binary_accuracy: 0.8539 - val_loss: 0.3015 - val_binary_accuracy: 0.8974
Epoch 2/250
11/11 [==============================] - 0s 7ms/step - loss: 0.3519 - binary_accuracy: 0.8669 - val_loss: 0.3096 - val_binary_accuracy: 0.9231
Epoch 3/250
11/11 [==============================] - 0s 7ms/step - loss: 0.3921 - binary_accuracy: 0.8799 - val_loss: 0.3162 - val_binary_accuracy: 0.9103
Epoch 4/250
11/11 [==============================] - 0s 7ms/step - loss: 0.3208 - binary_accuracy: 0.8864 - val_loss: 0.2732 - val_binary_accuracy: 0.9359
Epoch 5/250
11/11 [==============================] - 0s 9ms/step - loss: 0.3652 - binary_accuracy: 0.8669 - val_loss: 0.2659 - val_binary_accuracy: 0.9359
Epoch 6/250
11/11 [==============================] - 0s 7ms/step - loss: 0.3361 - binary_accuracy: 0.8929 - val_loss: 0.2654 - val_binary_accuracy: 0.9231
Epoch 7/250
11/11 [==============================] - 0s 7ms/step - lo

In [ ]:

#test with male only

model.fit(Total_X_train, Total_y_train, epochs = 250, 
          batch_size = 30, 
          validation_data = (Male_X_test, Male_y_test))

Epoch 1/250
11/11 [==============================] - 0s 10ms/step - loss: 0.2941 - binary_accuracy: 0.8831 - val_loss: 0.3242 - val_binary_accuracy: 0.8974
Epoch 2/250
11/11 [==============================] - 0s 7ms/step - loss: 0.2950 - binary_accuracy: 0.8864 - val_loss: 0.3327 - val_binary_accuracy: 0.8974
Epoch 3/250
11/11 [==============================] - 0s 7ms/step - loss: 0.2850 - binary_accuracy: 0.8994 - val_loss: 0.3339 - val_binary_accuracy: 0.9231
Epoch 4/250
11/11 [==============================] - 0s 8ms/step - loss: 0.2768 - binary_accuracy: 0.9026 - val_loss: 0.3494 - val_binary_accuracy: 0.9231
Epoch 5/250
11/11 [==============================] - 0s 6ms/step - loss: 0.3364 - binary_accuracy: 0.8734 - val_loss: 0.3560 - val_binary_accuracy: 0.9231
Epoch 6/250
11/11 [==============================] - 0s 7ms/step - loss: 0.3181 - binary_accuracy: 0.8766 - val_loss: 0.3385 - val_binary_accuracy: 0.9231
Epoch 7/250
11/11 [==============================] - 0s 7ms/step - lo

In [ ]:

#test with Female only

model.fit(Total_X_train, Total_y_train, epochs = 250, 
          batch_size = 30, 
          validation_data = (Female_X_test, Female_y_test))

Epoch 1/250
11/11 [==============================] - 0s 12ms/step - loss: 0.3542 - binary_accuracy: 0.8539 - val_loss: 0.2096 - val_binary_accuracy: 0.9487
Epoch 2/250
11/11 [==============================] - 0s 7ms/step - loss: 0.3320 - binary_accuracy: 0.8669 - val_loss: 0.2179 - val_binary_accuracy: 0.9487
Epoch 3/250
11/11 [==============================] - 0s 8ms/step - loss: 0.2958 - binary_accuracy: 0.8669 - val_loss: 0.2122 - val_binary_accuracy: 0.9487
Epoch 4/250
11/11 [==============================] - 0s 7ms/step - loss: 0.3368 - binary_accuracy: 0.8831 - val_loss: 0.2421 - val_binary_accuracy: 0.9487
Epoch 5/250
11/11 [==============================] - 0s 9ms/step - loss: 0.3636 - binary_accuracy: 0.8701 - val_loss: 0.2494 - val_binary_accuracy: 0.9231
Epoch 6/250
11/11 [==============================] - 0s 8ms/step - loss: 0.3306 - binary_accuracy: 0.8766 - val_loss: 0.2374 - val_binary_accuracy: 0.9487
Epoch 7/250
11/11 [==============================] - 0s 7ms/step - lo